In [181]:
import requests
from datetime import date
import json

In [195]:
def calc_exchange_rate():
    global calc_out
    global curr_exchg_date
    global curr_exchg_rate
    
    %store -r curr_exchg_date
    %store -r curr_exchg_rate
    
    if (curr_exchg_date == date.today()):
        calc_out = curr_exchg_rate
    else:
        calc_out = get_curr_exchg_rate()
    
    curr_exchg_date = date.today()
    curr_exchg_rate = calc_out
    
    %store calc_out
    %store curr_exchg_date
    %store curr_exchg_rate
    
#    print(" calc out: ", calc_out)
#    print("curr exchg date: ",curr_exchg_date)
#    print("curr exchg rate: ", curr_exchg_rate)

def get_curr_exchg_rate():
    global response_data
    url = 'https://open.er-api.com/v6/latest/USD'
    response_data = requests.get(url)
    %store response_data
    print(response_data)
    
    data = response_data.json()
    print(json.dumps(data, indent=4))
        
    resp_cd = data["result"]
    if resp_cd == "success":
        return data["rates"]["EUR"]
    else:
        return 0.9324
        
    
    
    

    

In [215]:
global curr_exchg_rate, calc_out
curr_exchg_rate = .934161
calc_out = .394161
%store curr_exchg_rate
%store calc_out

Stored 'curr_exchg_rate' (float)
Stored 'calc_out' (float)
